# Import needed libraries

In [4]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from sklearn.pipeline import make_pipeline
import category_encoders
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Activation, Dropout

import matplotlib.pylab as plt
%matplotlib inline

# Titanic dataset

## Data import and simple preprocessing.

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Preprocessing is taken from https://www.kaggle.com/abevallerian/titanic-with-tensorflow

In [0]:
#These columns are not useful for survive prediction
del train['Name']
del train['Ticket']
del train['Fare']
del train['Embarked']

train = train.fillna(value=0.0)

#Replacing sex with 0 (Female) or 1 (Male)
for i in range(train.shape[0]):
    if train.at[i, 'Sex'] == 'male':
        train.at[i, 'Sex'] = 1
    else:
        train.at[i, 'Sex'] = 0

#Since the age is categorical data, I group the age 8 groups: NaN, 0-10, 10-20, ..., 70-80.
# From the desribe above, it's shown that the maximum age is 80.
train['Age_group'] = 0
for i in range(train.shape[0]):
    for j in range(70, 0, -10):
        if train.at[i, 'Age'] > j:
            train.at[i, 'Age_group'] = int(j / 10)
            break
del train['Age']  # it's unnecessary anymore

#Cabin is quite interesting. It is stored in string.
# I think the format is written as Cabin Section + Cabin Number. I'm only interested in obtaining the Cabin Section.
train['Cabin_section'] = '0'
for i in range(train.shape[0]):
    if train.at[i, 'Cabin'] != 0:
        train.at[i, 'Cabin_section'] = train.at[i, 'Cabin'][0]
CABIN_SECTION = list(set(
    train['Cabin_section'].values))  # will be reused for test data
for i in range(train.shape[0]):
    train.at[i, 'Cabin_section'] = CABIN_SECTION.index(train.at[i, 'Cabin_section'])
del train['Cabin']  # it's unnecessary anymore

Converting data into numpy array

In [0]:
pclass = np.eye(train['Pclass'].values.max()+1)[train['Pclass'].values]
age_group = np.eye(train['Age_group'].values.max()+1)[train['Age_group'].values]
cabin_section = np.eye(train['Cabin_section'].values.max()+1) \
                    [train['Cabin_section'].values.astype(int)] # prevent IndexError

X = train[['Sex', 'SibSp', 'Parch']].values
X = np.concatenate([X, age_group], axis=1)
X = np.concatenate([X, pclass], axis=1)
X = np.concatenate([X, cabin_section], axis=1)
X = X.astype(float)

y = train['Survived'].values
y = y.astype(float).reshape(-1, 1)

#Dividing training data into training and validation (development) sets
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.1, random_state=0)

y_train = np.float32(y_train)

Same preprocessing for the test data

In [0]:
del test['Name']
del test['Ticket']
del test['Fare']
del test['Embarked']

test = test.fillna(value=0.0)

test['Age_group'] = 0
test['Cabin_section'] = '0'
for i in range(test.shape[0]):
    if test.at[i, 'Sex'] == 'male':
        test.at[i, 'Sex'] = 1
    else:
        test.at[i, 'Sex'] = 0

    for j in range(70, 0, -10):
        if test.at[i, 'Age'] > j:
            test.at[i, 'Age_group'] = int(j/10)
            break

    if test.at[i, 'Cabin'] != 0:
        test.at[i, 'Cabin_section'] = test.at[i, 'Cabin'][0]
    test.at[i, 'Cabin_section'] = CABIN_SECTION.index(test.at[i, 'Cabin_section'])

del test['Cabin'] # it's unnecessary anymore
del test['Age'] # it's unnecessary anymore


pclass_test = np.eye(test['Pclass'].values.max()+1)[test['Pclass'].values]
age_group_test = np.eye(test['Age_group'].values.max()+1)[test['Age_group'].values]
cabin_section_test = np.eye(test['Cabin_section'].values.max()+1) \
                    [test['Cabin_section'].values.astype(int)] # prevent IndexError

X_test = test[['Sex', 'SibSp', 'Parch']].values
X_test = np.concatenate([X_test, age_group_test], axis=1)
X_test = np.concatenate([X_test, pclass_test], axis=1)
X_test = np.concatenate([X_test, cabin_section_test], axis=1)
X_test = X_test.astype(float)

id_test = test['PassengerId'].values
id_test = id_test.reshape(-1, 1)

# Titanic classifcation with clear TensorFlow

In [0]:
tf.reset_default_graph()

N_EPOCHS = 10000
LEARNING_RATE = 0.001
input_size = X_train.shape[1]  #number of features
seed = 42

#Creating graph
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(seed)
    np.random.seed(seed)

    #Creating placeholders for X and Y input
    X_input = tf.placeholder(dtype=tf.float32,
                             shape=[None, input_size],
                             name='X_input')
    y_input = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='y_input')

    #Creating weight and bias, initialized from normal distribution
    w = tf.Variable(tf.random_normal(shape=[input_size, 1], seed=seed),
                    name='w')
    b = tf.Variable(tf.random_normal(shape=[1], seed=seed), name='b')

    #Building model to predict Y
    logits = tf.add(tf.matmul(X_input, w), b)
    sigm = tf.nn.sigmoid(logits, name='pred')
    pred = tf.cast(tf.greater_equal(sigm, 0.5), tf.float32,
                   name='pred')  # 1 if >= 0.5, 0 if < 0.5

    #Using cross enthropy loss function
    loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(labels=y_input,
                                                logits=logits,
                                                name='loss'))

    #Using Adam optimizer with learning rate of 0.001 to minimize loss
    train_steps = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

    #Calculating accuracy
    acc = tf.reduce_mean(tf.cast(tf.equal(pred, y_input), tf.float32),
                         name='acc')

    #Inializing all variables at once
    init_var = tf.global_variables_initializer()

#Feeding placeholders
train_feed_dict = {X_input: X_train, y_input: y_train}
dev_feed_dict = {X_input: X_dev, y_input: y_dev}
test_feed_dict = {
    X_input: X_test
}  # no y_input since the goal is to predict it

sess = tf.Session(graph=graph)
sess.run(init_var)

cur_loss = sess.run(loss, feed_dict=train_feed_dict)
train_acc = sess.run(acc, feed_dict=train_feed_dict)
dev_acc = sess.run(acc, feed_dict=dev_feed_dict)
print('step   0: loss {0:.5f}, train_acc {1:.2f}%, dev_acc {2:.2f}%'.format(
    cur_loss, 100 * train_acc, 100 * dev_acc))

for step in range(1, N_EPOCHS + 1):
    sess.run(train_steps, feed_dict=train_feed_dict)
    cur_loss = sess.run(loss, feed_dict=train_feed_dict)
    train_acc = sess.run(acc, feed_dict=train_feed_dict)
    dev_acc = sess.run(acc, feed_dict=dev_feed_dict)
    if step % 100 != 0:  # print result every 100 steps
        continue
    print(
        'step {3}: loss {0:.5f}, train_acc {1:.2f}%, dev_acc {2:.2f}%'.format(
            cur_loss, 100 * train_acc, 100 * dev_acc, step))

step   0: loss 1.76236, train_acc 35.96%, dev_acc 42.22%
step 100: loss 1.49557, train_acc 36.83%, dev_acc 41.11%
step 200: loss 1.28232, train_acc 37.83%, dev_acc 43.33%
step 300: loss 1.12105, train_acc 46.69%, dev_acc 52.22%
step 400: loss 1.00299, train_acc 47.82%, dev_acc 52.22%
step 500: loss 0.91789, train_acc 52.18%, dev_acc 52.22%
step 600: loss 0.85641, train_acc 51.56%, dev_acc 51.11%
step 700: loss 0.81103, train_acc 56.93%, dev_acc 57.78%
step 800: loss 0.77620, train_acc 57.43%, dev_acc 57.78%
step 900: loss 0.74816, train_acc 60.55%, dev_acc 64.44%
step 1000: loss 0.72459, train_acc 61.42%, dev_acc 64.44%
step 1100: loss 0.70413, train_acc 61.42%, dev_acc 64.44%
step 1200: loss 0.68602, train_acc 62.17%, dev_acc 65.56%
step 1300: loss 0.66982, train_acc 62.42%, dev_acc 64.44%
step 1400: loss 0.65523, train_acc 66.17%, dev_acc 67.78%
step 1500: loss 0.64205, train_acc 66.42%, dev_acc 67.78%
step 1600: loss 0.63008, train_acc 67.17%, dev_acc 68.89%
step 1700: loss 0.61914,

Making prediction on test set

In [0]:
y_pred = sess.run(pred, feed_dict=test_feed_dict).astype(int)
prediction = pd.DataFrame(np.concatenate([id_test, y_pred], axis=1),
                          columns=['PassengerId', 'Survived'])

In [0]:
prediction.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


# Titanic classification with Keras

In [0]:
# Initialising the NN
model = Sequential()

# layer
model.add(
    Dense(1,
          kernel_initializer='uniform',
          activation='sigmoid',
          input_dim=input_size))

# summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 25        
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [0]:
N_EPOCHS = 1000
LEARNING_RATE = 0.001

#initializing optimizer
adam_opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compiling the NN
model.compile(optimizer = adam_opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the NN
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), batch_size = 32, epochs = N_EPOCHS)

Train on 801 samples, validate on 90 samples
Epoch 1/1000
801/801 [==============================] - 0s 169us/step - loss: 0.6950 - acc: 0.4694 - val_loss: 0.6795 - val_acc: 0.7444
Epoch 2/1000
801/801 [==============================] - 0s 100us/step - loss: 0.6778 - acc: 0.6592 - val_loss: 0.6653 - val_acc: 0.7000
Epoch 3/1000
801/801 [==============================] - 0s 103us/step - loss: 0.6638 - acc: 0.6604 - val_loss: 0.6539 - val_acc: 0.6667
Epoch 4/1000
801/801 [==============================] - 0s 92us/step - loss: 0.6543 - acc: 0.6542 - val_loss: 0.6450 - val_acc: 0.6778
Epoch 5/1000
801/801 [==============================] - 0s 86us/step - loss: 0.6447 - acc: 0.6617 - val_loss: 0.6361 - val_acc: 0.6667
Epoch 6/1000
801/801 [==============================] - 0s 87us/step - loss: 0.6364 - acc: 0.6692 - val_loss: 0.6283 - val_acc: 0.6556
Epoch 7/1000
801/801 [==============================] - 0s 76us/step - loss: 0.6290 - acc: 0.6742 - val_loss: 0.6210 - val_acc: 0.6889
Epoch 8

Epoch 61/1000
801/801 [==============================] - 0s 93us/step - loss: 0.5126 - acc: 0.7815 - val_loss: 0.4790 - val_acc: 0.8000
Epoch 62/1000
801/801 [==============================] - 0s 87us/step - loss: 0.5115 - acc: 0.7828 - val_loss: 0.4784 - val_acc: 0.8111
Epoch 63/1000
801/801 [==============================] - 0s 86us/step - loss: 0.5104 - acc: 0.7840 - val_loss: 0.4771 - val_acc: 0.8111
Epoch 64/1000
801/801 [==============================] - 0s 88us/step - loss: 0.5096 - acc: 0.7840 - val_loss: 0.4762 - val_acc: 0.8111
Epoch 65/1000
801/801 [==============================] - 0s 109us/step - loss: 0.5085 - acc: 0.7840 - val_loss: 0.4757 - val_acc: 0.8111
Epoch 66/1000
801/801 [==============================] - 0s 128us/step - loss: 0.5076 - acc: 0.7840 - val_loss: 0.4750 - val_acc: 0.8222
Epoch 67/1000
801/801 [==============================] - 0s 112us/step - loss: 0.5068 - acc: 0.7853 - val_loss: 0.4738 - val_acc: 0.8111
Epoch 68/1000
801/801 [======================

Epoch 121/1000
801/801 [==============================] - 0s 92us/step - loss: 0.4736 - acc: 0.7965 - val_loss: 0.4369 - val_acc: 0.8222
Epoch 122/1000
801/801 [==============================] - 0s 67us/step - loss: 0.4734 - acc: 0.8015 - val_loss: 0.4367 - val_acc: 0.8222
Epoch 123/1000
801/801 [==============================] - 0s 77us/step - loss: 0.4731 - acc: 0.8152 - val_loss: 0.4362 - val_acc: 0.8222
Epoch 124/1000
801/801 [==============================] - 0s 76us/step - loss: 0.4727 - acc: 0.8027 - val_loss: 0.4356 - val_acc: 0.8111
Epoch 125/1000
801/801 [==============================] - 0s 81us/step - loss: 0.4724 - acc: 0.7990 - val_loss: 0.4347 - val_acc: 0.8222
Epoch 126/1000
801/801 [==============================] - 0s 77us/step - loss: 0.4720 - acc: 0.7978 - val_loss: 0.4343 - val_acc: 0.8000
Epoch 127/1000
801/801 [==============================] - 0s 75us/step - loss: 0.4717 - acc: 0.8015 - val_loss: 0.4345 - val_acc: 0.8222
Epoch 128/1000
801/801 [=================

Epoch 181/1000
801/801 [==============================] - 0s 85us/step - loss: 0.4577 - acc: 0.8152 - val_loss: 0.4123 - val_acc: 0.8333
Epoch 182/1000
801/801 [==============================] - 0s 83us/step - loss: 0.4574 - acc: 0.8165 - val_loss: 0.4120 - val_acc: 0.8333
Epoch 183/1000
801/801 [==============================] - 0s 73us/step - loss: 0.4573 - acc: 0.8152 - val_loss: 0.4120 - val_acc: 0.8333
Epoch 184/1000
801/801 [==============================] - 0s 78us/step - loss: 0.4572 - acc: 0.8152 - val_loss: 0.4115 - val_acc: 0.8333
Epoch 185/1000
801/801 [==============================] - 0s 76us/step - loss: 0.4570 - acc: 0.8165 - val_loss: 0.4116 - val_acc: 0.8333
Epoch 186/1000
801/801 [==============================] - 0s 86us/step - loss: 0.4568 - acc: 0.8165 - val_loss: 0.4113 - val_acc: 0.8333
Epoch 187/1000
801/801 [==============================] - 0s 76us/step - loss: 0.4568 - acc: 0.8165 - val_loss: 0.4111 - val_acc: 0.8333
Epoch 188/1000
801/801 [=================

Epoch 241/1000
801/801 [==============================] - 0s 82us/step - loss: 0.4512 - acc: 0.8202 - val_loss: 0.4073 - val_acc: 0.8333
Epoch 242/1000
801/801 [==============================] - 0s 92us/step - loss: 0.4512 - acc: 0.8215 - val_loss: 0.4072 - val_acc: 0.8333
Epoch 243/1000
801/801 [==============================] - 0s 155us/step - loss: 0.4512 - acc: 0.8227 - val_loss: 0.4069 - val_acc: 0.8444
Epoch 244/1000
801/801 [==============================] - 0s 108us/step - loss: 0.4512 - acc: 0.8227 - val_loss: 0.4068 - val_acc: 0.8444
Epoch 245/1000
801/801 [==============================] - 0s 85us/step - loss: 0.4510 - acc: 0.8227 - val_loss: 0.4065 - val_acc: 0.8444
Epoch 246/1000
801/801 [==============================] - 0s 90us/step - loss: 0.4510 - acc: 0.8227 - val_loss: 0.4064 - val_acc: 0.8444
Epoch 247/1000
801/801 [==============================] - 0s 108us/step - loss: 0.4510 - acc: 0.8227 - val_loss: 0.4062 - val_acc: 0.8444
Epoch 248/1000
801/801 [==============

...

Epoch 659/1000
801/801 [==============================] - 0s 82us/step - loss: 0.4465 - acc: 0.8127 - val_loss: 0.4011 - val_acc: 0.8444
Epoch 660/1000
801/801 [==============================] - 0s 95us/step - loss: 0.4466 - acc: 0.8127 - val_loss: 0.4018 - val_acc: 0.8444
Epoch 661/1000
801/801 [==============================] - 0s 108us/step - loss: 0.4465 - acc: 0.8115 - val_loss: 0.4016 - val_acc: 0.8444
Epoch 662/1000
801/801 [==============================] - 0s 75us/step - loss: 0.4465 - acc: 0.8127 - val_loss: 0.4015 - val_acc: 0.8444
Epoch 663/1000
801/801 [==============================] - 0s 69us/step - loss: 0.4466 - acc: 0.8115 - val_loss: 0.4014 - val_acc: 0.8444
Epoch 664/1000
801/801 [==============================] - 0s 88us/step - loss: 0.4465 - acc: 0.8115 - val_loss: 0.4015 - val_acc: 0.8444
Epoch 665/1000
801/801 [==============================] - 0s 79us/step - loss: 0.4466 - acc: 0.8102 - val_loss: 0.4024 - val_acc: 0.8444
Epoch 666/1000
801/801 [================

Epoch 719/1000
801/801 [==============================] - 0s 84us/step - loss: 0.4466 - acc: 0.8040 - val_loss: 0.4018 - val_acc: 0.8333
Epoch 720/1000
801/801 [==============================] - 0s 75us/step - loss: 0.4466 - acc: 0.8065 - val_loss: 0.4015 - val_acc: 0.8333
Epoch 721/1000
801/801 [==============================] - 0s 94us/step - loss: 0.4466 - acc: 0.8115 - val_loss: 0.4013 - val_acc: 0.8444
Epoch 722/1000
801/801 [==============================] - 0s 84us/step - loss: 0.4467 - acc: 0.8040 - val_loss: 0.4014 - val_acc: 0.8333
Epoch 723/1000
801/801 [==============================] - 0s 90us/step - loss: 0.4469 - acc: 0.8052 - val_loss: 0.4017 - val_acc: 0.8333
Epoch 724/1000
801/801 [==============================] - 0s 85us/step - loss: 0.4468 - acc: 0.8052 - val_loss: 0.4014 - val_acc: 0.8333
Epoch 725/1000
801/801 [==============================] - 0s 82us/step - loss: 0.4468 - acc: 0.8077 - val_loss: 0.4013 - val_acc: 0.8333
Epoch 726/1000
801/801 [=================

Epoch 779/1000
801/801 [==============================] - 0s 100us/step - loss: 0.4467 - acc: 0.8102 - val_loss: 0.4007 - val_acc: 0.8444
Epoch 780/1000
801/801 [==============================] - 0s 113us/step - loss: 0.4467 - acc: 0.8102 - val_loss: 0.4005 - val_acc: 0.8444
Epoch 781/1000
801/801 [==============================] - 0s 80us/step - loss: 0.4466 - acc: 0.8102 - val_loss: 0.4004 - val_acc: 0.8444
Epoch 782/1000
801/801 [==============================] - 0s 81us/step - loss: 0.4465 - acc: 0.8102 - val_loss: 0.4001 - val_acc: 0.8444
Epoch 783/1000
801/801 [==============================] - 0s 105us/step - loss: 0.4467 - acc: 0.8102 - val_loss: 0.4010 - val_acc: 0.8444
Epoch 784/1000
801/801 [==============================] - 0s 100us/step - loss: 0.4465 - acc: 0.8090 - val_loss: 0.4010 - val_acc: 0.8444
Epoch 785/1000
801/801 [==============================] - 0s 79us/step - loss: 0.4465 - acc: 0.8090 - val_loss: 0.4009 - val_acc: 0.8444
Epoch 786/1000
801/801 [=============

Epoch 898/1000
801/801 [==============================] - 0s 136us/step - loss: 0.4464 - acc: 0.8090 - val_loss: 0.4010 - val_acc: 0.8444
Epoch 899/1000
801/801 [==============================] - 0s 145us/step - loss: 0.4465 - acc: 0.8102 - val_loss: 0.4014 - val_acc: 0.8444
Epoch 900/1000
801/801 [==============================] - 0s 128us/step - loss: 0.4466 - acc: 0.8102 - val_loss: 0.4013 - val_acc: 0.8444
Epoch 901/1000
801/801 [==============================] - 0s 88us/step - loss: 0.4465 - acc: 0.8102 - val_loss: 0.4011 - val_acc: 0.8444
Epoch 902/1000
801/801 [==============================] - 0s 82us/step - loss: 0.4465 - acc: 0.8102 - val_loss: 0.4011 - val_acc: 0.8444
Epoch 903/1000
801/801 [==============================] - 0s 164us/step - loss: 0.4465 - acc: 0.8102 - val_loss: 0.4011 - val_acc: 0.8444
Epoch 904/1000
801/801 [==============================] - 0s 143us/step - loss: 0.4465 - acc: 0.8090 - val_loss: 0.4014 - val_acc: 0.8444
Epoch 905/1000
801/801 [============

Epoch 958/1000
801/801 [==============================] - 0s 71us/step - loss: 0.4467 - acc: 0.8077 - val_loss: 0.4011 - val_acc: 0.8444
Epoch 959/1000
801/801 [==============================] - 0s 81us/step - loss: 0.4467 - acc: 0.8040 - val_loss: 0.4017 - val_acc: 0.8333
Epoch 960/1000
801/801 [==============================] - 0s 84us/step - loss: 0.4465 - acc: 0.8015 - val_loss: 0.4020 - val_acc: 0.8333
Epoch 961/1000
801/801 [==============================] - 0s 69us/step - loss: 0.4465 - acc: 0.8015 - val_loss: 0.4018 - val_acc: 0.8333
Epoch 962/1000
801/801 [==============================] - 0s 83us/step - loss: 0.4465 - acc: 0.8015 - val_loss: 0.4017 - val_acc: 0.8333
Epoch 963/1000
801/801 [==============================] - 0s 81us/step - loss: 0.4465 - acc: 0.8015 - val_loss: 0.4016 - val_acc: 0.8333
Epoch 964/1000
801/801 [==============================] - 0s 78us/step - loss: 0.4465 - acc: 0.8015 - val_loss: 0.4015 - val_acc: 0.8333
Epoch 965/1000
801/801 [=================

Making predictions

In [0]:
y_pred = model.predict(X_test)
y_final = (y_pred > 0.5).astype(int).reshape(X_test.shape[0],1)

prediction = pd.DataFrame(np.concatenate([id_test, y_final], axis=1),
                          columns=['PassengerId', 'Survived'])

In [0]:
prediction.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


# Thyroid Desiase dataset 

## Data import and preprocessing

Let's take these steps from previous work (HW4)

In [0]:
all_data=pd.read_csv('dataset_57_hypothyroid.csv')
display(all_data.shape)

(3772, 30)

In [0]:
# Replacing ? with nan
all_data = all_data.replace('?', np.nan)
# Deleting variables with constant values
all_data = all_data.drop(['TBG', 'TBG_measured', 'hypopituitary'], 1)

# Converting categorical variables to numeric
to_numeric = ['FTI', 'T3', 'T4U', 'TSH', 'TT4', 'age']
for cat in to_numeric:
    all_data[cat] = pd.to_numeric(all_data[cat])

# Converting categorical variables into binary
all_data = all_data.replace('t', 1)
all_data = all_data.replace('f', 0)
all_data['sex'] = all_data['sex'].replace('M', 1)
all_data['sex'] = all_data['sex'].replace('F', 0)
all_data = all_data.rename(index=str, columns={"sex": "Male"})
pipeline = make_pipeline(
    category_encoders.OneHotEncoder(handle_unknown="ignore",
                                    use_cat_names=True,
                                    cols=['referral_source']))
all_data = pipeline.fit_transform(all_data)

# Deleting useless variables
all_data = all_data[all_data.columns.drop(
    list(all_data.filter(regex='measured')))]
# Deleting NA
all_data = all_data.dropna()
# Converting Male into binary
all_data['Male'] = all_data['Male'].astype(np.int64)

# Selecting numeric variables
numeric_feats = all_data.dtypes[all_data.dtypes == "float64"].index
# Numeric variables processing
all_data = all_data[all_data['age'] < 400]
all_data = all_data.drop('T4U', 1)
numeric_feats = numeric_feats.drop('T4U', 1)
# Selecting binary variables
binary_feats = all_data.columns
for feat in numeric_feats:
    binary_feats = binary_feats.drop(feat, 1)

# Deleting variables with equal ratio for sick and healthy people
# Deleting variables with constant values
all_data = all_data.drop([
    'referral_source_other', 'referral_source_SVI', 'referral_source_SVHD',
    'query_on_thyroxine', 'sick', 'query_hyperthyroid', 'lithium', 'tumor',
    'goitre', 'pregnant'
], 1)
binary_feats = binary_feats.drop([
    'referral_source_other', 'referral_source_SVI', 'referral_source_SVHD',
    'query_on_thyroxine', 'sick', 'query_hyperthyroid', 'lithium', 'tumor',
    'goitre', 'pregnant'
], 1)

# Target selecting
target = all_data.pop('Class')
binary_feats = binary_feats.drop('Class', 1)

all_data = all_data.dropna()

ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_sample(all_data, target)

In [0]:
# Converting to DataFrame
X_resampled_pd = pd.DataFrame(data=X_resampled, columns=all_data.columns)
# Converting non-binary variables which sholud be binary
X_resampled_pd[binary_feats] = X_resampled_pd[binary_feats].astype(np.int64)

display(X_resampled_pd.shape, X_resampled_pd.head())

y_resampled_pd = pd.Series(data=y_resampled, name='Class')

display(y_resampled_pd.shape, y_resampled_pd.head())

(9704, 14)

,referral_source_SVHC,referral_source_STMW,age,Male,on_thyroxine,on_antithyroid_medication,thyroid_surgery,I131_treatment,query_hypothyroid,psych,TSH,T3,TT4,FTI
0,1,0,41.0,0,0,0,0,0,0,0,1.30,2.5,125.0,109.0
1,0,0,70.0,0,0,0,0,0,0,0,0.72,1.2,61.0,70.0
2,0,0,80.0,0,0,0,0,0,0,0,2.20,0.6,80.0,115.0
3,0,0,66.0,0,0,0,0,0,0,0,0.60,2.2,123.0,132.0
4,0,0,68.0,1,0,0,0,0,0,0,2.40,1.6,83.0,93.0


(9704,)

0    negative
1    negative
2    negative
3    negative
4    negative
Name: Class, dtype: object

Let's provide target in one-hot form

In [0]:
num_labels = len(set(y_resampled_pd))

In [0]:
Counter(y_resampled_pd)

Counter({'compensated_hypothyroid': 2426,
         'negative': 2426,
         'primary_hypothyroid': 2426,
         'secondary_hypothyroid': 2426})

In [0]:
y_resampled_pd.replace('negative', 0, inplace=True)
y_resampled_pd.replace('primary_hypothyroid', 1, inplace=True)
y_resampled_pd.replace('secondary_hypothyroid', 2, inplace=True)
y_resampled_pd.replace('compensated_hypothyroid', 3, inplace=True)

In [0]:
# One-hot for y
y_resampled_pd = (np.arange(num_labels) == np.array(y_resampled_pd)[:,None]).astype(np.float32)

In [0]:
X_train_dev, X_test, y_train_dev, y_test = train_test_split(X_resampled_pd,
                                                            y_resampled_pd,
                                                            test_size=0.15,
                                                            random_state=0)
print('X_train_dev: ', X_train_dev.shape, '\nX_test: ', X_test.shape,
      '\ny_train_dev: ', y_train_dev.shape, '\ny_test: ', y_test.shape)

X_train_dev:  (8248, 14) 
X_test:  (1456, 14) 
y_train_dev:  (8248, 4) 
y_test:  (1456, 4)


In [0]:
X_train, X_dev, y_train, y_dev = train_test_split(X_train_dev,
                                                  y_train_dev,
                                                  test_size=0.2,
                                                  random_state=0)
print('X_train: ', X_train.shape, '\nX_dev: ', X_dev.shape, '\ny_train: ',
      y_train.shape, '\ny_dev: ', y_dev.shape)

X_train:  (6598, 14) 
X_dev:  (1650, 14) 
y_train:  (6598, 4) 
y_dev:  (1650, 4)


In [0]:
X_train = X_train.values.astype(float)
X_dev = X_dev.values.astype(float)
X_test = X_test.values.astype(float)

# Thyroid Desiase classification with clear TensorFlow

In [0]:
tf.reset_default_graph()

N_EPOCHS = 10000
LEARNING_RATE = 0.001
input_size = X_train.shape[1]  #number of features
seed = 42

#Creating graph
graph = tf.Graph()
with graph.as_default():
    tf.set_random_seed(seed)
    np.random.seed(seed)

    #Creating placeholders for X and Y input
    X_input = tf.placeholder(dtype=tf.float32,
                             shape=[None, input_size],
                             name='X_input')
    y_input = tf.placeholder(dtype=tf.float32,
                             shape=[None, num_labels],
                             name='y_input')

    #Creating weight and bias, initialized from normal distribution
    w = tf.Variable(tf.random_normal(shape=[input_size, num_labels],
                                     seed=seed),
                    name='w')
    b = tf.Variable(tf.random_normal(shape=[num_labels], seed=seed), name='b')

    #Building model to predict Y
    logits = tf.add(tf.matmul(X_input, w), b)
    pred = tf.nn.softmax(logits)

    #Using cross enthropy loss function
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_input,
                                                logits=logits,
                                                name='loss'))

    #Using Adam optimizer with learning rate of 0.001 to minimize loss
    train_steps = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

    #Calculating accuracy
    acc = tf.reduce_mean(tf.cast(tf.equal(np.argmax(pred), np.argmax(y_input)),
                                 tf.float32),
                         name='acc')

    #Inializing all variables at once
    init_var = tf.global_variables_initializer()

#Feeding placeholders
train_feed_dict = {X_input: X_train, y_input: y_train}
dev_feed_dict = {X_input: X_dev, y_input: y_dev}
test_feed_dict = {
    X_input: X_test
}  # no y_input since the goal is to predict it

sess = tf.Session(graph=graph)
sess.run(init_var)

#Initial run
cur_loss = sess.run(loss, feed_dict=train_feed_dict)
train_acc = sess.run(acc, feed_dict=train_feed_dict)
dev_acc = sess.run(acc, feed_dict=dev_feed_dict)
print('step   0: loss {0:.5f}, train_acc {1:.2f}%, dev_acc {2:.2f}%'.format(
    cur_loss, 100 * train_acc, 100 * dev_acc))

for step in range(1, N_EPOCHS + 1):
    sess.run(train_steps, feed_dict=train_feed_dict)

    cur_loss = sess.run(loss, feed_dict=train_feed_dict)
    train_acc = sess.run(acc, feed_dict=train_feed_dict)
    dev_acc = sess.run(acc, feed_dict=dev_feed_dict)
    if step % 100 != 0:  # print result every 100 steps
        continue
    print(
        'step {3}: loss {0:.5f}, train_acc {1:.2f}%, dev_acc {2:.2f}%'.format(
            cur_loss, 100 * train_acc, 100 * dev_acc, step))

step   0: loss 205.99641, train_acc 100.00%, dev_acc 100.00%
step 100: loss 175.45625, train_acc 100.00%, dev_acc 100.00%
step 200: loss 151.16364, train_acc 100.00%, dev_acc 100.00%
step 300: loss 128.14554, train_acc 100.00%, dev_acc 100.00%
step 400: loss 105.42755, train_acc 100.00%, dev_acc 100.00%
step 500: loss 86.00930, train_acc 100.00%, dev_acc 100.00%
step 600: loss 70.67134, train_acc 100.00%, dev_acc 100.00%
step 700: loss 61.84097, train_acc 100.00%, dev_acc 100.00%
step 800: loss 55.88947, train_acc 100.00%, dev_acc 100.00%
step 900: loss 50.19123, train_acc 100.00%, dev_acc 100.00%
step 1000: loss 44.52027, train_acc 100.00%, dev_acc 100.00%
step 1100: loss 38.82970, train_acc 100.00%, dev_acc 100.00%
step 1200: loss 33.21935, train_acc 100.00%, dev_acc 100.00%
step 1300: loss 27.81915, train_acc 100.00%, dev_acc 100.00%
step 1400: loss 22.77228, train_acc 100.00%, dev_acc 100.00%
step 1500: loss 17.78319, train_acc 100.00%, dev_acc 100.00%
step 1600: loss 13.40028, tra

Making prediction on test set

In [0]:
y_pred = sess.run(pred, feed_dict=test_feed_dict)

In [0]:
y_pred[:5]

array([[9.4432153e-23, 9.9975628e-01, 5.7006046e-19, 2.4366759e-04],
       [1.4105912e-02, 2.8582759e-02, 9.3752563e-01, 1.9785790e-02],
       [1.0000000e+00, 1.4310562e-13, 3.3855738e-17, 1.9260706e-10],
       [2.0475909e-19, 9.9998355e-01, 2.3275499e-09, 1.6477563e-05],
       [2.6482583e-31, 9.9996090e-01, 4.9270070e-18, 3.9072256e-05]],
      dtype=float32)

In [0]:
y_test[:5]

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [0]:
correct = 0
for i in range(len(y_pred)):
    if(np.argmax(y_pred[i])==np.argmax(y_test[i])):
        correct+=1
print('test accuracy is ',correct/len(y_test))

test accuracy is  0.9629120879120879


# Thyroid Desiase classification with Keras

In [0]:
input_size = X_train.shape[1]

# Initialising the NN
model = Sequential()

# layer
model.add(
    Dense(num_labels,
          kernel_initializer='uniform',
          activation='softmax',
          input_dim=input_size))

# summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 60        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________


In [0]:
N_EPOCHS = 100
LEARNING_RATE = 0.001

#initializing optimizer
adam_opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# Compiling the NN
model.compile(optimizer = adam_opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Train the NN
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), batch_size = 32, epochs = N_EPOCHS)

Train on 6598 samples, validate on 1650 samples
Epoch 1/100
6598/6598 [==============================] - 1s 134us/step - loss: 1.1249 - acc: 0.6023 - val_loss: 0.7528 - val_acc: 0.8182
Epoch 2/100
6598/6598 [==============================] - 1s 90us/step - loss: 0.6511 - acc: 0.8454 - val_loss: 0.6325 - val_acc: 0.8145
Epoch 3/100
6598/6598 [==============================] - 1s 89us/step - loss: 0.5513 - acc: 0.8731 - val_loss: 0.5601 - val_acc: 0.8582
Epoch 4/100
6598/6598 [==============================] - 1s 90us/step - loss: 0.4917 - acc: 0.8844 - val_loss: 0.5108 - val_acc: 0.8824
Epoch 5/100
6598/6598 [==============================] - 1s 92us/step - loss: 0.4462 - acc: 0.9033 - val_loss: 0.4767 - val_acc: 0.8800
Epoch 6/100
6598/6598 [==============================] - 1s 89us/step - loss: 0.4130 - acc: 0.9075 - val_loss: 0.4538 - val_acc: 0.8976
Epoch 7/100
6598/6598 [==============================] - 1s 91us/step - loss: 0.3842 - acc: 0.9174 - val_loss: 0.4286 - val_acc: 0.9048

Making prediction on test set

In [0]:
y_pred = model.predict(X_test)

In [0]:
y_pred[:5]

array([[7.9889335e-17, 5.2069753e-01, 5.0540625e-06, 4.7929737e-01],
       [7.0900400e-04, 1.1808803e-02, 2.7189169e-02, 9.6029299e-01],
       [1.0000000e+00, 5.5427488e-14, 1.2631550e-14, 3.0414964e-09],
       [2.6823562e-20, 8.2109475e-01, 3.7917489e-10, 1.7890523e-01],
       [0.0000000e+00, 9.9883240e-01, 1.4575347e-37, 1.1675935e-03]],
      dtype=float32)

In [0]:
y_test[:5]

array([[0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [0]:
correct = 0
for i in range(len(y_pred)):
    if(np.argmax(y_pred[i])==np.argmax(y_test[i])):
        correct+=1
print('test accuracy is ',correct/len(y_test))

test accuracy is  0.9608516483516484
